# Creating Stored Procedures in Snowpark

Operationalize Snowpark stored Procedures

    Create Stored Procedures from files

    Write Python Functions to power stored procedures

    Register Stored Procedures

    Data types(type hints vs registration API)

For more information follow the below links:

1. [Creating Stored Procedures for DataFrames in Python](https://docs.snowflake.com/en/developer-guide/snowpark/python/creating-sprocs)

2. [Operationalizing your code with Snowpark Python Stored Procedures](https://medium.com/snowflake/operationalizing-your-code-with-snowpark-python-stored-procedures-a16c77afcff)
    

In [ ]:
import pandas as pd
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.use_database("snowpark_db")
session.use_schema("sourced")
session.query_tag = "create-sps"

In [ ]:
#Anonymous SP(with Sproc + Lambda)
add_one = sproc(
    lambda session, x: session.sql(f"select {x} + 1").collect()[0][0],
    input_types=[IntegerType()],
    return_type=IntegerType(),
    packages=["snowflake-snowpark-python"])

ret = add_one(1)
print(f"add_one: {ret}")

In [ ]:
#Names SP(with Sproc + lambda)
add_two =  sproc(
    lambda session, x:session.sql(f"select {x} +2").collect()[0][0],
    input_types = [IntegerType()],
    return_type=IntegerType(),
    name="add_two_proc",
    replace=True,
    packages=["snowflake-snowpark-python"])

ret = session.call("add_two_proc",1)
print(f"add_two:{ret}")

In [ ]:
#registered SP(with @proc decorator)
@sproc(name ="add_three",
    replace = True,
    is_permanent= True,
    stage_location="@snowpark_db.sourced.stage1",
       packages=["snowflake-snowpark-python"]
      )

def add_three(session:Session, x:int) -> int:
    return session.sql(f"select {x} + 3").collect()[0][0]

ret = session.sql("call add_three(1)").collect()[0][0]
print(f"add_three: {ret}")

In [ ]:
# registered SP (with @sproc decorator + add_packages)
session.add_packages("snowflake-snowpark-python", "pandas")
# session.add_import(...)
# session.add_requirements(...)

@sproc(name="showVersion",
    replace=True,
    is_permanent=True,
    stage_location="@snowpark_db.sourced.stage1")
def showVersion(session: Session) -> str:
   return pd.__version__

ret = session.call("showVersion")
print(f"showVersion: {ret}")

In [ ]:
#Registered SP(with @sproc decorator + add_requirments)
#must upload requirments.txt in this notebook

session.clear_packages()
session.add_requirements('requirments.txt')

@sproc(name="showVersion"
      ,replace= True
      ,is_permanent=True
      ,stage_location="@snowpark_db.sourced.stage1"
      )
def showVersion(session:Session) -> str:
    return pd.__version__

ret = session.call("showVersion")
print(f"showVersion: {ret}")

In [ ]:
# registered SP (with register + inline lambda)
session.sproc.register(
    lambda session, x: session.sql(f"select {x} + 4").collect()[0][0],
    name="add_four",
    input_types=[IntegerType()],
    return_type=IntegerType(),
    replace=True,
    is_permanent=True,
    packages=["snowflake-snowpark-python"],
    stage_location="@stage1")

ret = session.sql("call add_four(1)").collect()[0][0]
print(f"add_four: {ret}")

In [ ]:
# registered SP (from uploaded local Python file)
# must upload demo16.py file in this notebook!
add_five = session.sproc.register_from_file(
    file_path="demo16.py",
    func_name="add_five",
    packages=["snowflake-snowpark-python"])

print(f"add_five: {add_five(1)}")

In [ ]:
# registered SP (from staged Python file)
# must upload demo16.py file in this notebook!
session.file.put("demo16.py", "@stage1", auto_compress=False)

add_five = session.sproc.register_from_file(
    file_path="@stage1/demo16.py",
    func_name="add_five",
    return_type=IntegerType(),
    input_types=[IntegerType()],
    packages=["snowflake-snowpark-python"])

print(f"add_five: {add_five(1)}")